In [1]:
import geopandas as gp
path = 'data/dc_lts/DC_LTS_Final.shp'
lts_gpdf = gp.read_file(path)
lts_gpdf_bfr = lts_gpdf.copy()
lts_gpdf_bfr.geometry = lts_gpdf_bfr.buffer(10)
lts_gpdf_bfr = lts_gpdf_bfr.to_crs(epsg=4326)
segs_gpdf = gp.read_file('data/segments_dc.geojson')

In [2]:
dc_seg_bfr = segs_gpdf[['STREETSEGID','geometry','SHAPE_Length','STREETTYPE','SEGMENTTYPE']].to_crs(crs=lts_gpdf.crs)
dc_seg_bfr.geometry = dc_seg_bfr.buffer(10)
dc_seg_bfr = dc_seg_bfr.to_crs(epsg=4326)

In [3]:
def get_area_percentage(x):
    area_left = x.geometry_x.area
    area_right = x.geometry_y.area
    return x.area/area_left if area_left<area_right else x.area/area_right

import pandas as pd

def get_overlap(left_gdf, right_gdf):
    jgdf = gp.tools.sjoin(left_gdf, right_gdf, how='left')
#     jgdf = jgdf.rename(columns={'geometry':'geometry_left'})
    jgdf_intrsetn = pd.merge(jgdf,right_gdf[['geometry']],left_on='index_right',right_index = True)
    print jgdf_intrsetn.columns
    jgdf_intrsetn['intersection']=jgdf_intrsetn.apply(lambda x: x.geometry_x.intersection(x.geometry_y),axis=1)
    jgdf_intrsetn['area'] = jgdf_intrsetn['intersection'].apply(lambda x: x.area)
    jgdf_intrsetn['area_prcnt'] = jgdf_intrsetn.apply(get_area_percentage, axis=1)
    return jgdf_intrsetn

In [4]:
overlap = get_overlap(dc_seg_bfr,lts_gpdf_bfr)
# bk_st_ovlp = bk_st_bfr_intrsetn[bk_st_bfr_intrsetn.area_prcnt>0.9]

Index([ u'STREETSEGID',   u'geometry_x', u'SHAPE_Length',   u'STREETTYPE',
        u'SEGMENTTYPE',  u'index_right',          u'LTS',    u'Length_Ft',
           u'LgthLTS1',     u'LgthLTS2',     u'LgthLTS3',     u'LgthLTS4',
          u'RoadClass',         u'Ward',   u'geometry_y'],
      dtype='object')


In [5]:
match_lts = overlap[overlap.area_prcnt>0.8]

In [17]:
lts_segid = match_lts.groupby(u'STREETSEGID')['LTS'].apply(max)

In [18]:
lts_segid_df = lts_segid.to_frame().reset_index()

In [19]:
df = segs_gpdf[['STREETSEGID']].merge(lts_segid_df, how='left')
df.index.name = 'index_seg'

In [23]:
df.drop("STREETSEGID", axis=1).to_csv('data/feature_lts_dc.csv')

# visualization

In [1]:
import geopandas as gp
path = 'data/dc_lts/DC_LTS_Final.shp'
gpdf = gp.read_file(path)

In [2]:
# gpdf.Ward.plot(kind='bar')
gpdf.Ward.value_counts()

4    3658
5    3411
3    3172
7    3085
6    3076
2    2560
8    2462
1    1322
dtype: int64

In [3]:
colors_for_lts = {1:'#9CE81E',2:'#14FFC8', 3:'#FFA800', 4:'#E8130C', 10:'#6B0DFF'}
gpdf_crs = gpdf.to_crs(epsg=4326)
gpdf_crs['color'] = gpdf_crs.LTS.apply(lambda x: colors_for_lts[x])


In [91]:
gpdfs = []
binding_data = []
for i in [1,2,3,4,10]:
    gpdfs.append(gpdf_crs[gpdf_crs.LTS==i])
    binding_data.append(['LTS_{}'.format(i), 'LTS={}'.format(i)])

In [92]:
from utils.leaflet_creation_v2 import create_map_visualization


import utils.leaflet_creation_v2;reload(utils.leaflet_creation_v2)
from utils.leaflet_creation_v2 import get_color_for_df, create_map_visualization
html_title = 'dc_ls'
file_path = ''
file_name = 'dc_ls'
lon, lat = -77.0163424758, 38.9047829846
zoom = 12
init_layer = ['light']
map_layer = ['light','streets','satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layer, map_layer, binding_data, gpdfs)

In [3]:
%matplotlib inline

In [8]:
gpdf_bfr = gpdf.copy()
gpdf_bfr.geometry = gpdf_bfr.buffer(10)
gpdf_bfr = gpdf_bfr.to_crs(epsg=4326)

In [9]:
dc_seg_path = 'data/opendc_segments.geojson'
dc_seg = gp.read_file(dc_seg_path)

In [10]:
dc_seg_bfr = dc_seg[['STREETSEGID','geometry','SHAPE_Length','STREETTYPE','SEGMENTTYPE']].to_crs(crs=gpdf.crs)
dc_seg_bfr

,STREETSEGID,geometry,SHAPE_Length,STREETTYPE,SEGMENTTYPE
0,876,LINESTRING (1308809.198557231 467678.249573956...,192.925131,ST,2
1,13161,LINESTRING (1309023.830696209 468605.905392078...,131.935677,ST,2
2,8240,LINESTRING (1309042.334590227 469033.791762678...,117.887011,ST,2
3,10395,LINESTRING (1309030.228303999 469420.339624164...,108.331118,ST,2
4,2216,LINESTRING (1309029.933020851 469775.752371289...,119.190312,ST,2
5,10090,LINESTRING (1309029.834586938 470166.794975653...,119.101173,ST,2
6,8384,LINESTRING (1309031.245336672 470557.542510008...,78.878075,ST,2
7,11625,LINESTRING (1309030.326697264 470816.301682450...,79.410694,ST,2
8,7886,LINESTRING (1309026.660285117 474341.000277610...,95.591087,ST,2
9,5660,LINESTRING (1309015.595665408 474651.859093696...,157.511599,ST,2


In [11]:
dc_seg_bfr.geometry = dc_seg_bfr.buffer(10)
dc_seg_bfr = dc_seg_bfr.to_crs(epsg=4326)

In [12]:
def get_area_percentage(x):
    area_left = x.geometry_x.area
    area_right = x.geometry_y.area
    return x.area/area_left if area_left<area_right else x.area/area_right

import pandas as pd

def get_overlap(left_gdf, right_gdf):
    jgdf = gp.tools.sjoin(left_gdf, right_gdf, how='left')
#     jgdf = jgdf.rename(columns={'geometry':'geometry_left'})
    jgdf_intrsetn = pd.merge(jgdf,right_gdf[['geometry']],left_on='index_right',right_index = True)
    print jgdf_intrsetn.columns
    jgdf_intrsetn['intersection']=jgdf_intrsetn.apply(lambda x: x.geometry_x.intersection(x.geometry_y),axis=1)
    jgdf_intrsetn['area'] = jgdf_intrsetn['intersection'].apply(lambda x: x.area)
    jgdf_intrsetn['area_prcnt'] = jgdf_intrsetn.apply(get_area_percentage, axis=1)
    return jgdf_intrsetn

In [13]:
overlap = get_overlap(dc_seg_bfr,gpdf_bfr)
# bk_st_ovlp = bk_st_bfr_intrsetn[bk_st_bfr_intrsetn.area_prcnt>0.9]

Index([ u'STREETSEGID',   u'geometry_x', u'SHAPE_Length',   u'STREETTYPE',
        u'SEGMENTTYPE',  u'index_right',          u'LTS',    u'Length_Ft',
           u'LgthLTS1',     u'LgthLTS2',     u'LgthLTS3',     u'LgthLTS4',
          u'RoadClass',         u'Ward',   u'geometry_y'],
      dtype='object')


In [14]:
overlap

,STREETSEGID,geometry_x,SHAPE_Length,STREETTYPE,SEGMENTTYPE,index_right,LTS,Length_Ft,LgthLTS1,LgthLTS2,LgthLTS3,LgthLTS4,RoadClass,Ward,geometry_y,intersection,area,area_prcnt
0,876,POLYGON ((-77.01237887789435 38.95090759113995...,192.925131,ST,2,3944,2,454.524184,0.000000,454.524184,0.000000,0,Street,4,POLYGON ((-77.01320652277667 38.94978159124903...,POLYGON ((-77.01235474361909 38.95083132000458...,2.503364e-09,0.027572
86,4761,POLYGON ((-77.01320880053403 38.94978939222759...,138.539918,ST,2,3944,2,454.524184,0.000000,454.524184,0.000000,0,Street,4,POLYGON ((-77.01320652277667 38.94978159124903...,POLYGON ((-77.01235691245141 38.95083826274116...,8.146754e-08,0.897291
3098,8078,POLYGON ((-77.01319843223143 38.94973338748601...,85.718698,ST,2,3944,2,454.524184,0.000000,454.524184,0.000000,0,Street,4,POLYGON ((-77.01320652277667 38.94978159124903...,POLYGON ((-77.01319843223143 38.94973338748601...,2.864860e-09,0.049972
3267,1429,POLYGON ((-77.01248624311442 38.95084484330926...,85.018355,ST,2,3944,2,454.524184,0.000000,454.524184,0.000000,0,Street,4,POLYGON ((-77.01320652277667 38.94978159124903...,POLYGON ((-77.01235690065433 38.95083824096557...,2.590815e-09,0.045542
0,876,POLYGON ((-77.01237887789435 38.95090759113995...,192.925131,ST,2,506,2,131.986368,0.000000,131.986368,0.000000,0,Street,4,POLYGON ((-77.01248395894797 38.95083704636153...,POLYGON ((-77.01235473057606 38.95083133731854...,2.919125e-09,0.102374
86,4761,POLYGON ((-77.01320880053403 38.94978939222759...,138.539918,ST,2,506,2,131.986368,0.000000,131.986368,0.000000,0,Street,4,POLYGON ((-77.01248395894797 38.95083704636153...,POLYGON ((-77.01244462665304 38.95082036995431...,3.581551e-09,0.125605
3267,1429,POLYGON ((-77.01248624311442 38.95084484330926...,85.018355,ST,2,506,2,131.986368,0.000000,131.986368,0.000000,0,Street,4,POLYGON ((-77.01248395894797 38.95083704636153...,POLYGON ((-77.01248624311442 38.95084484330926...,2.524681e-08,0.885406
0,876,POLYGON ((-77.01237887789435 38.95090759113995...,192.925131,ST,2,8058,2,252.061955,0.000000,252.061955,0.000000,0,Street,4,POLYGON ((-77.01237659309027 38.95089979389922...,POLYGON ((-77.01191006028503 38.95143016072586...,4.632149e-08,0.895954
86,4761,POLYGON ((-77.01320880053403 38.94978939222759...,138.539918,ST,2,8058,2,252.061955,0.000000,252.061955,0.000000,0,Street,4,POLYGON ((-77.01237659309027 38.95089979389922...,POLYGON ((-77.01240552244927 38.95086139159721...,3.239293e-09,0.062655
3267,1429,POLYGON ((-77.01248624311442 38.95084484330926...,85.018355,ST,2,8058,2,252.061955,0.000000,252.061955,0.000000,0,Street,4,POLYGON ((-77.01237659309027 38.95089979389922...,POLYGON ((-77.01239883387318 38.95087027035318...,2.962480e-09,0.057301


In [15]:
overlap.area_prcnt.hist()

In [16]:
a = overlap[overlap.area_prcnt>0.8].STREETSEGID.value_counts()
a[a>1].shape, a[a==1].shape, a[a>1].shape[0]+a[a==1].shape[0], dc_seg.shape

((5704L,), (7649L,), 13353L, (13522, 22))

In [17]:
multi_match_seg = set(a[a>2].index.tolist())
two_match_seg = set(a[a==2].index.tolist())
one_match_seg = set(a[a==1].index.tolist())
no_match_seg = set(dc_seg.STREETSEGID)-set(a.index.tolist())

In [18]:
len(multi_match_seg), len(two_match_seg), len(one_match_seg), len(no_match_seg)

(1932, 3772, 7649, 169)

In [19]:
169.0/13522

0.012498151161070847

In [20]:
def get_ls(seg_ids, joined,ls):
    return ls.loc[pd.unique(joined[(joined.STREETSEGID.isin(seg_ids))&(joined.area_prcnt>0.8)].index_right)]
ls = get_ls(multi_match_seg,overlap,gpdf_bfr)

In [25]:
seg_ids = [multi_match_seg, two_match_seg,
           one_match_seg, no_match_seg
          ]
colors = ['#f00', '#0f0', '#00f', '#ff0', '#f0f']
binding_data = [
    ['multi_match_seg','multi_match_seg'],
    ['multi_match_seg_ls','multi_match_seg_ls'],
    ['two_match_seg','two_match_seg'],
    ['two_match_seg_ls','two_match_seg_ls'],
    ['one_match_seg','one_match_seg'],
    ['one_match_seg_ls','one_match_seg_ls'],
    ['no_match_seg','no_match_seg'],
    ['no_match_seg_ls','no_match_seg_ls'],
]

gpdfs = []
for i, sids in enumerate(seg_ids):
    gggggpdf = dc_seg[dc_seg.STREETSEGID.isin(sids)].copy()
    gggggpdf['color'] = colors[i]
    gpdfs.append(gggggpdf)
    ls_gpdf = get_ls(sids, overlap, gpdf_bfr).copy()
    ls_gpdf['color'] = ls_gpdf.LTS.apply(lambda x: colors_for_lts[x])

    ls_gpdf.reset_index(inplace=True)
    gpdfs.append(ls_gpdf)

In [26]:
from utils.leaflet_creation_v2 import create_map_visualization


import utils.leaflet_creation_v2;reload(utils.leaflet_creation_v2)
from utils.leaflet_creation_v2 import get_color_for_df, create_map_visualization
html_title = 'dc_seg_ls'
file_path = ''
file_name = 'dc_seg_ls'
lon, lat = -77.0163424758, 38.9047829846
zoom = 12
init_layer = ['light']
map_layer = ['light','streets','satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layer, map_layer, binding_data, gpdfs)